# Export InSAR data

## Tools

*Modules*

In [39]:
import numpy as np
from osgeo import gdal, osr
import matplotlib.pyplot as plt
import datetime

In [2]:
gdal.UseExceptions()
gdal.SetCacheMax(32)

*Paths*

In [3]:
# paths
wkdir = "/Users/emile/Documents/Etude/2024_2025_M2/tutored_project"
datadir = "{:}/data".format(wkdir)
plotdir = "{:}/figures".format(wkdir)

*Parameters*

In [4]:
rad2cm = - 5.5465763 / (4*np.pi)

crs_geo = 'EPSG:4326'
crs_utm = 'EPSG:32636'

In [ ]:
from pyproj import Transformer

## Load InSAR velocities

*Ascending track*

In [5]:
foldername = 'NSBAS_TS-PKG_S1_LEVANT-A087-VV-2015-2021_IW123_2015-03-04_2021-04-25'

In [6]:
# Mean LOS velocity

data_ = gdal.OpenEx("{:}/InSAR_FLATSIM/{:}/CNES_MV-LOS_geo_8rlks.tiff".format(datadir, foldername), 
                    nOpenFlags=gdal.GA_ReadOnly)

ny = data_.RasterYSize
nx = data_.RasterXSize
nband = data_.RasterCount
no_data_value = data_.GetRasterBand(1).GetNoDataValue()

insar_asc_vel = data_.GetRasterBand(1).ReadAsArray(0, 0, nx, ny) * rad2cm
insar_asc_mask_nodata = insar_asc_vel == no_data_value

In [7]:
# Lon / lat coordinates (EPSG:4326)

xoff, a, b, yoff, d, e = data_.GetGeoTransform()
old_cs = osr.SpatialReference()
old_cs.ImportFromWkt(data_.GetProjectionRef())
new_cs = osr.SpatialReference()
new_cs.ImportFromEPSG(4326)
transform = osr.CoordinateTransformation(old_cs, new_cs)

x_ = np.arange(nx)
y_ = np.arange(ny)
X, Y = np.meshgrid(x_, y_)

x = a*X + b*Y + xoff
y = d*X + e*Y + yoff

insar_asc_lon = np.zeros_like(insar_asc_vel)
insar_asc_lat = np.zeros_like(insar_asc_vel)

for i in range(ny):
    for j in range(nx):
        lon_, lat_, _ = transform.TransformPoint(x[i, j], y[i, j])
        
        insar_asc_lon[i, j] = lon_
        insar_asc_lat[i, j] = lat_

In [8]:
# X / Y coordinates UTM (EPSG:32636)

xoff, a, b, yoff, d, e = data_.GetGeoTransform()
old_cs = osr.SpatialReference()
old_cs.ImportFromWkt(data_.GetProjectionRef())
new_cs = osr.SpatialReference()
new_cs.ImportFromEPSG(32636)
transform = osr.CoordinateTransformation(old_cs, new_cs)

x_ = np.arange(nx)
y_ = np.arange(ny)
X, Y = np.meshgrid(x_, y_)

x = a*X + b*Y + xoff
y = d*X + e*Y + yoff

insar_asc_x = np.zeros_like(insar_asc_vel)
insar_asc_y = np.zeros_like(insar_asc_vel)

for i in range(ny):
    for j in range(nx):
        x__, y__, __ = transform.TransformPoint(y[i, j], x[i, j])

        insar_asc_x[i, j] = x__
        insar_asc_y[i, j] = y__

del data_

In [9]:
# LOS

data_ = gdal.OpenEx("{:}/InSAR_FLATSIM/{:}/CNES_CosENU_geo_8rlks.tiff".format(datadir, foldername),
                    nOpenFlags=gdal.GA_ReadOnly)

insar_asc_LOS = np.zeros((ny, nx, 3))
insar_asc_LOS[:, :, 0] = data_.GetRasterBand(1).ReadAsArray(0, 0, nx, ny)
insar_asc_LOS[:, :, 1] = data_.GetRasterBand(2).ReadAsArray(0, 0, nx, ny)
insar_asc_LOS[:, :, 2] = data_.GetRasterBand(3).ReadAsArray(0, 0, nx, ny)
insar_asc_LOS_mean = np.mean(insar_asc_LOS, axis=(0, 1))

del data_

In [10]:
# Nb interferograms, misclosure, temporal coherence

data_ = gdal.OpenEx("{:}/InSAR_FLATSIM/{:}/CNES_Net_geo_8rlks.tiff".format(datadir, foldername),
                    nOpenFlags=gdal.GA_ReadOnly)

insar_asc_nbinterf = data_.GetRasterBand(2).ReadAsArray(0, 0, nx, ny)
insar_asc_misclosure = data_.GetRasterBand(1).ReadAsArray(0, 0, nx, ny)
insar_asc_coherence = data_.GetRasterBand(4).ReadAsArray(0, 0, nx, ny) / 16

del data_

*Descending track*

In [11]:
foldername = 'NSBAS_TS-PKG_S1_LEVANT-D021SUD-VV-2014-2021_IW123_2014-10-31_2021-04-27'

In [12]:
# Mean LOS velocity

data_ = gdal.OpenEx("{:}/InSAR_FLATSIM/{:}/CNES_MV-LOS_geo_8rlks.tiff".format(datadir, foldername), 
                    nOpenFlags=gdal.GA_ReadOnly)

ny = data_.RasterYSize
nx = data_.RasterXSize
nband = data_.RasterCount
no_data_value = data_.GetRasterBand(1).GetNoDataValue()

insar_desc_vel = data_.GetRasterBand(1).ReadAsArray(0, 0, nx, ny) * rad2cm
insar_desc_mask_nodata = insar_desc_vel == no_data_value

In [13]:
# Lon / lat coordinates (EPSG:4326)

xoff, a, b, yoff, d, e = data_.GetGeoTransform()
old_cs = osr.SpatialReference()
old_cs.ImportFromWkt(data_.GetProjectionRef())
new_cs = osr.SpatialReference()
new_cs.ImportFromEPSG(4326)
transform = osr.CoordinateTransformation(old_cs, new_cs)

x_ = np.arange(nx)
y_ = np.arange(ny)
X, Y = np.meshgrid(x_, y_)

x = a*X + b*Y + xoff
y = d*X + e*Y + yoff

insar_desc_lon = np.zeros_like(insar_desc_vel)
insar_desc_lat = np.zeros_like(insar_desc_vel)

for i in range(ny):
    for j in range(nx):
        lon_, lat_, _ = transform.TransformPoint(x[i, j], y[i, j])
        
        insar_desc_lon[i, j] = lon_
        insar_desc_lat[i, j] = lat_

In [14]:
# X / Y coordinates UTM (EPSG:32636)

xoff, a, b, yoff, d, e = data_.GetGeoTransform()
old_cs = osr.SpatialReference()
old_cs.ImportFromWkt(data_.GetProjectionRef())
new_cs = osr.SpatialReference()
new_cs.ImportFromEPSG(32636)
transform = osr.CoordinateTransformation(old_cs, new_cs)

x_ = np.arange(nx)
y_ = np.arange(ny)
X, Y = np.meshgrid(x_, y_)

x = a*X + b*Y + xoff
y = d*X + e*Y + yoff

insar_desc_x = np.zeros_like(insar_desc_vel)
insar_desc_y = np.zeros_like(insar_desc_vel)

for i in range(ny):
    for j in range(nx):
        x__, y__, __ = transform.TransformPoint(y[i, j], x[i, j])
        
        insar_desc_x[i, j] = x__
        insar_desc_y[i, j] = y__

del data_

In [15]:
# LOS

data_ = gdal.OpenEx("{:}/InSAR_FLATSIM/{:}/CNES_CosENU_geo_8rlks.tiff".format(datadir, foldername),
                    nOpenFlags=gdal.GA_ReadOnly)

insar_desc_LOS = np.zeros((ny, nx, 3))
insar_desc_LOS[:, :, 0] = data_.GetRasterBand(1).ReadAsArray(0, 0, nx, ny)
insar_desc_LOS[:, :, 1] = data_.GetRasterBand(2).ReadAsArray(0, 0, nx, ny)
insar_desc_LOS[:, :, 2] = data_.GetRasterBand(3).ReadAsArray(0, 0, nx, ny)
insar_desc_LOS_mean = np.mean(insar_desc_LOS, axis=(0, 1))

del data_

In [16]:
# Nb interferograms, misclosure, temporal coherence

data_ = gdal.OpenEx("{:}/InSAR_FLATSIM/{:}/CNES_Net_geo_8rlks.tiff".format(datadir, foldername),
                    nOpenFlags=gdal.GA_ReadOnly)

insar_desc_nbinterf = data_.GetRasterBand(2).ReadAsArray(0, 0, nx, ny)
insar_desc_misclosure = data_.GetRasterBand(1).ReadAsArray(0, 0, nx, ny)
insar_desc_coherence = data_.GetRasterBand(4).ReadAsArray(0, 0, nx, ny) / 16

del data_

## Draft: correct plate motion model

In [ ]:
############################################################
# Program is part of MintPy                                #
# Copyright (c) 2013, Zhang Yunjun, Heresh Fattahi         #
# Author: Yuan-Kai Liu, Zhang Yunjun, May 2022             #
############################################################

# Earth radius
# equatorial radius: a = 6378.1370e3
# polar      radius: b = 6356.7523e3
# arithmetic mean radius: R_1 = (2 * a + b) / 3 = 6371.0088e3
#   defined by IUGG and used in geophysics
EARTH_RADIUS = 6371.0088e3   # the arithmetic mean radius in meters

MAS2RAD = np.pi / 3600000 / 180    # 1 mas (milli arc second) = x radian
MASY2DMY = 1e6 / 3600000           # 1 mas per year = x degree per million year


def cart2sph(rx, ry, rz):
    """Convert cartesian coordinates to spherical.

    Parameters: rx/y/z  - float / np.ndarray, angular distance in X/Y/Z direction [any units of distance]
    Returns:    lat/lon - float / np.ndarray, latitude / longitude  [degree]
                r       - float / np.ndarray, radius [same unit as rx/y/z]
    Examples:
        # convert xyz coord to spherical coord
        lat, lon, r = cart2sph(x, y, z)
        # convert Euler vector (in cartesian) to Euler pole (in spherical)
        pole_lat, pole_lon, rot_rate = cart2sph(wx, wy, wz)
    """
    r = np.sqrt(rx**2 + ry**2 + rz**2)
    lat = np.rad2deg(np.arcsin(rz / r))
    lon = np.rad2deg(np.arctan2(ry, rx))
    return lat, lon, r


def sph2cart(lat, lon, r=1):
    """Convert spherical coordinates to cartesian.

    Parameters: lat/lon - float / np.ndarray, latitude / longitude [degree]
                r       - float / np.ndarray, radius [any units of angular distance]
    Returns:    rx/y/z  - float / np.ndarray, angular distance in X/Y/Z direction [same unit as r]
    Examples:
        # convert spherical coord to xyz coord
        x, y, z = sph2cart(lat, lon, r=radius)
        # convert Euler pole (in spherical) to Euler vector (in cartesian)
        wx, wy, wz = sph2cart(pole_lat, pole_lon, r=rot_rate)
    """
    rx = r * np.cos(np.deg2rad(lat)) * np.cos(np.deg2rad(lon))
    ry = r * np.cos(np.deg2rad(lat)) * np.sin(np.deg2rad(lon))
    rz = r * np.sin(np.deg2rad(lat))
    return rx, ry, rz


def transform_xyz_enu(lat, lon, x=None, y=None, z=None, e=None, n=None, u=None):
    """Transform between ECEF (global xyz) and ENU at given locations (lat, lon) via matrix rotation.

    Reference:
        Navipedia, https://gssc.esa.int/navipedia/index.php/Transformations_between_ECEF_and_ENU_coordinates
        Cox, A., and Hart, R.B. (1986) Plate tectonics: How it works. Blackwell Scientific Publications,
          Palo Alto, doi: 10.4236/ojapps.2015.54016. Page 145-156

    Parameters: lat/lon - float / np.ndarray, latitude/longitude      at location(s) [degree]
                x/y/z   - float / np.ndarray, x/y/z         component at location(s) [e.g., length, velocity]
                e/n/u   - float / np.ndarray, east/north/up component at location(s) [e.g., length, velocity]
    Returns:    e/n/u if given x/y/z
                x/y/z if given e/n/u
    """
    # convert the unit from degree to radian
    lat = np.deg2rad(lat)
    lon = np.deg2rad(lon)

    # transformation via matrix rotation:
    #     V_enu = T * V_xyz
    #     V_xyz = T^-1 * V_enu
    #
    # Equilevant 3D matrix code is as below:
    #     V_enu = np.diagonal(
    #         np.matmul(
    #             T.reshape([-1,3]),
    #             V_xyz.T,
    #         ).reshape([3, npts, npts], order='F'),
    #         axis1=1,
    #         axis2=2,
    #     ).T
    # To avoid this complex matrix operation above, we calculate for each element as below:

    if all(i is not None for i in [x, y, z]):
        # cart2enu
        e = - np.sin(lon) * x \
            + np.cos(lon) * y
        n = - np.sin(lat) * np.cos(lon) * x \
            - np.sin(lat) * np.sin(lon) * y \
            + np.cos(lat) * z
        u =   np.cos(lat) * np.cos(lon) * x \
            + np.cos(lat) * np.sin(lon) * y \
            + np.sin(lat) * z
        return e, n, u

    elif all(i is not None for i in [e, n, u]):
        # enu2cart
        x = - np.sin(lon) * e \
            - np.cos(lon) * np.sin(lat) * n \
            + np.cos(lon) * np.cos(lat) * u
        y =   np.cos(lon) * e \
            - np.sin(lon) * np.sin(lat) * n \
            + np.sin(lon) * np.cos(lat) * u
        z =   np.cos(lat) * n \
            + np.sin(lat) * u
        return x, y, z

    else:
        raise ValueError('Input (x,y,z) or (e,n,u) is NOT complete!')


def coord_llh2xyz(lat, lon, alt):
    """Convert coordinates from WGS84 lat/long/hgt to ECEF x/y/z.

    Parameters: lat   - float / list(float) / np.ndarray, latitude  [degree]
                lon   - float / list(float) / np.ndarray, longitude [degree]
                alt   - float / list(float) / np.ndarray, altitude  [meter]
    Returns:    x/y/z - float / list(float) / np.ndarray, ECEF coordinate [meter]
    """
    # ensure same type between alt and lat/lon
    if isinstance(lat, np.ndarray) and not isinstance(alt, np.ndarray):
        alt *= np.ones_like(lat)
    elif isinstance(lat, list) and not isinstance(alt, list):
        alt = [alt] * len(lat)

    # construct pyproj transform object
    transformer = pyproj.Transformer.from_crs(
        {"proj":'latlong', "ellps":'WGS84', "datum":'WGS84'},
        {"proj":'geocent', "ellps":'WGS84', "datum":'WGS84'},
    )

    # apply coordinate transformation
    x, y, z = transformer.transform(lon, lat, alt, radians=False)

    return x, y, z


def euler_pole_get_velocity_xyz(wx, wy, wz, lat, lon, alt=0.0, ellps=True):
    """Compute cartesian velocity (vx, vy, vz) of the Euler Pole at point(s) of interest.

    Parameters: wx/xy/wz - floats, Euler rotation vector ocomponent (mas/yr)    
                lat   - float / 1D/2D np.ndarray, points of interest (latitude)  [degree]
                lon   - float / 1D/2D np.ndarray, points of interest (longitude) [degree]
                alt   - float / 1D/2D np.ndarray, points of interest (altitude)      [meter]
                ellps - bool, consider ellipsoidal Earth projection
    Returns:    vx    - float / 1D/2D np.ndarray, ECEF x linear velocity [meter/year]
                vy    - float / 1D/2D np.ndarray, ECEF y linear velocity [meter/year]
                vz    - float / 1D/2D np.ndarray, ECEF z linear velocity [meter/year]
    """
    # check input lat/lon data type (scalar / array) and shape
    poi_shape = lat.shape if isinstance(lat, np.ndarray) else None

    # convert lat/lon into x/y/z
    # Note: the conversion assumes either a spherical or spheroidal Earth, tests show that
    # using a ellipsoid as defined in WGS84 produce results closer to the UNAVCO website
    # calculator, which also uses the WGS84 ellipsoid.
    if ellps:
        x, y, z = coord_llh2xyz(lat, lon, alt)
    else:
        x, y, z = sph2cart(lat, lon, alt+EARTH_RADIUS)

    # ensure matrix is flattened
    if poi_shape is not None:
        x = x.flatten()
        y = y.flatten()
        z = z.flatten()

    # compute the cartesian linear velocity (i.e., ECEF) in meter/year as:
    #
    #     V_xyz = Omega x R_i
    #
    # where R_i is location vector at point i

    xyz = np.array([x, y, z], dtype=np.float32)
    omega = np.array([wx, wy, wz]) * MAS2RAD
    vx, vy, vz = np.cross(omega, xyz.T).T.reshape(xyz.shape)

    # reshape to the original shape of lat/lon
    if poi_shape is not None:
        vx = vx.reshape(poi_shape)
        vy = vy.reshape(poi_shape)
        vz = vz.reshape(poi_shape)

    return vx, vy, vz


def euler_pole_get_velocity_enu(wx, wy, wz, lat, lon, alt=0.0, ellps=True):
        """Compute the spherical velocity (ve, vn, vu) of the Euler Pole at point(s) of interest.

        Parameters: wx/xy/wz - floats, Euler rotation vector ocomponent (mas/yr)    
                    lat   - float / 1D/2D np.ndarray, points of interest (latitude)  [degree]
                    lon   - float / 1D/2D np.ndarray, points of interest (longitude) [degree]
                    alt   - float / 1D/2D np.ndarray, points of interest (altitude) [meter]
                    ellps - bool, consider ellipsoidal Earth projection
        Returns:    ve    - float / 1D/2D np.ndarray, east  linear velocity [meter/year]
                    vn    - float / 1D/2D np.ndarray, north linear velocity [meter/year]
                    vu    - float / 1D/2D np.ndarray, up    linear velocity [meter/year]
        """
        # calculate ECEF velocity
        vx, vy, vz = euler_pole_get_velocity_xyz(wx, wy, wz, lat, lon, alt=alt, ellps=ellps)

        # convert ECEF to ENU velocity via matrix rotation: V_enu = T * V_xyz
        ve, vn, vu = transform_xyz_enu(lat, lon, x=vx, y=vy, z=vz)

        # enforce zero vertical velocitpy when ellps=False
        # to avoid artifacts due to numerical precision
        if not ellps:
            if isinstance(lat, np.ndarray):
                vu[:] = 0
            else:
                vu = 0

        return ve, vn, vu

In [ ]:
ITRF2014_PMM = {'ARAB': [1.154, -0.136, 1.444, 0.515], # omega_x (mas/yr) omega_y (mas/yr) omega_z (mas/yr) omega (deg/Ma)
                'NUBI': [0.099, -0.614, 0.733, 0.267]}

wx, wy, wz = ITRF2014_PMM['ARAB'][:3]

In [ ]:
pmm_asc = euler_pole_get_velocity_enu(wx=wx, wy=wy, wz=wz,
                                      lat=insar_asc_lat, lon=insar_asc_lon)
pmm_asc = np.stack(pmm_asc, axis=-1)

insar_pmm_asc = - np.sum(pmm_asc * insar_asc_LOS, axis=2)
insar_asc_vel_pmm = 1e3 * insar_pmm_asc
insar_asc_vel_pmm[insar_asc_mask_nodata] = np.nan

## Export InSAR velocities

*Ascending track*

In [17]:
np.save("{:}/InSAR_FLATSIM/numpy_format/insar_asc_vel.npy".format(datadir), insar_asc_vel)
np.save("{:}/InSAR_FLATSIM/numpy_format/insar_asc_mask_nodata.npy".format(datadir), insar_asc_mask_nodata)
np.save("{:}/InSAR_FLATSIM/numpy_format/insar_asc_LOS.npy".format(datadir), insar_asc_LOS)

np.save("{:}/InSAR_FLATSIM/numpy_format/insar_asc_misclosure.npy".format(datadir), insar_asc_misclosure)
np.save("{:}/InSAR_FLATSIM/numpy_format/insar_asc_nbinterf.npy".format(datadir), insar_asc_nbinterf)
np.save("{:}/InSAR_FLATSIM/numpy_format/insar_asc_coherence.npy".format(datadir), insar_asc_coherence)

np.save("{:}/InSAR_FLATSIM/numpy_format/insar_asc_lat.npy".format(datadir), insar_asc_lat)
np.save("{:}/InSAR_FLATSIM/numpy_format/insar_asc_lon.npy".format(datadir), insar_asc_lon)

np.save("{:}/InSAR_FLATSIM/numpy_format/insar_asc_x_utm.npy".format(datadir), insar_asc_x)
np.save("{:}/InSAR_FLATSIM/numpy_format/insar_asc_y_utm.npy".format(datadir), insar_asc_y)

*Descending track*

In [18]:
np.save("{:}/InSAR_FLATSIM/numpy_format/insar_desc_vel.npy".format(datadir), insar_desc_vel)
np.save("{:}/InSAR_FLATSIM/numpy_format/insar_desc_mask_nodata.npy".format(datadir), insar_desc_mask_nodata)
np.save("{:}/InSAR_FLATSIM/numpy_format/insar_desc_LOS.npy".format(datadir), insar_desc_LOS)

np.save("{:}/InSAR_FLATSIM/numpy_format/insar_desc_misclosure.npy".format(datadir), insar_desc_misclosure)
np.save("{:}/InSAR_FLATSIM/numpy_format/insar_desc_nbinterf.npy".format(datadir), insar_desc_nbinterf)
np.save("{:}/InSAR_FLATSIM/numpy_format/insar_desc_coherence.npy".format(datadir), insar_desc_coherence)

np.save("{:}/InSAR_FLATSIM/numpy_format/insar_desc_lat.npy".format(datadir), insar_desc_lat)
np.save("{:}/InSAR_FLATSIM/numpy_format/insar_desc_lon.npy".format(datadir), insar_desc_lon)

np.save("{:}/InSAR_FLATSIM/numpy_format/insar_desc_x_utm.npy".format(datadir), insar_desc_x)
np.save("{:}/InSAR_FLATSIM/numpy_format/insar_desc_y_utm.npy".format(datadir), insar_desc_y)

## Load InSAR pixel time series

*Ascending track*

In [35]:
foldername = 'NSBAS_TS-PKG_S1_LEVANT-A087-VV-2015-2021_IW123_2015-03-04_2021-04-25'

In [37]:
data_ = gdal.OpenEx("{:}/InSAR_FLATSIM/{:}/CNES_cube_geo_8rlks.tiff".format(datadir, foldername), 
                    nOpenFlags=gdal.GA_ReadOnly)

ny = data_.RasterYSize
nx = data_.RasterXSize
nband = data_.RasterCount

insar_asc_time = [datetime.datetime.strptime(data_.GetRasterBand(k).GetDescription()[14:], "%Y%m%d")
                  for k in range(1, nband+1)]

In [73]:
lonpix = 35.3
latpix = 31.6
dpix = 5

ipix = np.argmin(np.abs(insar_asc_lon[0, :] - lonpix))
jpix = np.argmin(np.abs(insar_asc_lat[:, 0] - latpix))

In [74]:
disp = np.array([])

for k in range(1, nband+1):
    disp = np.concatenate((disp, [np.nanmean(data_.GetRasterBand(k).ReadAsArray(jpix-(dpix-1)/2, ipix-(dpix-1)/2,
                                                                                dpix, dpix))]))
    
disp *= rad2cm

In [24]:
del data_

*Descending track*

In [25]:
foldername = 'NSBAS_TS-PKG_S1_LEVANT-D021SUD-VV-2014-2021_IW123_2014-10-31_2021-04-27'

In [26]:
data_ = gdal.OpenEx("{:}/InSAR_FLATSIM/{:}/CNES_cube_geo_8rlks.tiff".format(datadir, foldername), 
                    nOpenFlags=gdal.GA_ReadOnly)

ny = data_.RasterYSize
nx = data_.RasterXSize
nband = data_.RasterCount

In [27]:
del data_

## Export InSAR pixel time series

*Ascending track*

*Descending track*